In [6]:
%%timeit -n 10
import numpy as np
import math
import cv2
                # we will read the image unchanged
img = cv2.imread('field 1-3.jpg', cv2.IMREAD_UNCHANGED) 
roi = img[600:1800, 50:440]
roi = cv2.GaussianBlur(roi,(3,3),0)
roi2 = roi.copy()
roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
roi = cv2.Canny(roi,100,550,3)

pixel_to_metric = 134.0596881
initial_pole =  464.0387914819
intial_angle = 89.26
lines = cv2.HoughLinesP(roi,1,np.pi/180,100,maxLineGap = 100) #Hough transformation. 
no_of_lines = len(lines)

edges = []              #LINE SEPERATION
angles= []
lengths = []
for i in range(no_of_lines):
    if ((lines[i][0][3]-lines[i][0][1]) != 0 and (lines[i][0][2]-lines[i][0][0]) != 0):
        ang = abs(math.degrees(math.atan((lines[i][0][3]-lines[i][0][1])/(lines[i][0][2]-lines[i][0][0]))))
        if (abs(90 - ang) <= 5):
            edges.append(lines[i][0])
            angles.append(ang)
            lengths.append(math.sqrt((lines[i][0][2]-lines[i][0][0])**2 + (lines[i][0][3]-lines[i][0][1])**2))
    elif ((lines[i][0][2]-lines[i][0][0]) == 0):
        edges.append(lines[i][0])
        angles.append(90)
        lengths.append(math.sqrt((lines[i][0][2]-lines[i][0][0])**2 + (lines[i][0][3]-lines[i][0][1])**2))
temp=angles.copy()

for i in range(len(edges)):        #LINES DRAWN
    cv2.line(roi2, (edges[i][0],edges[i][1]), (edges[i][2],edges[i][3]), (0,255,0), 1)

index = [angles.index(max(angles))]    # 2 LINES CLOSER TWO 90 DEGREES 
temp[index[0]] = 0
index.append(temp.index(max(temp)))

                                         # START AND END POINTS
if (abs(lengths[0]-lengths[1]) <= 20):    
    start_point = (int((edges[index[0]][0]+edges[index[1]][0])/2), int((edges[index[0]][1]+edges[index[1]][1])/2))
    end_point = (int((edges[index[0]][2]+edges[index[1]][2])/2), int((edges[index[0]][3]+edges[index[1]][3])/2))
else:
    start_point = (edges[lengths.index(max(lengths))][0],edges[lengths.index(max(lengths))][1])
    end_point = (edges[lengths.index(max(lengths))][2], edges[lengths.index(max(lengths))][3])
cv2.line(roi2, start_point, end_point, (0,0,255), 1)

               # SNOW LEVEL CALCULATIONS.
pole_length = ((start_point[0]-end_point[0])**2 + (start_point[1]-end_point[1])**2)**0.5
snow_level = initial_pole - pole_length
snow_level_metric = snow_level/pixel_to_metric  #angle of the pole is neglected because only edges with 90 +- 5 is taken and in worst case sin(85) = 0.99
snow_level_metric = round(snow_level_metric,5)
#print("snow_level_metric:",snow_level_metric)

cv2.putText(roi2, str(snow_level_metric), (30,434), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
cv2.putText(roi2, "mts", (30,454), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

#cv2.namedWindow('roi',cv2.WINDOW_NORMAL)
#cv2.imshow('roi',roi2)

#cv2.waitKey(0)

#cv2.imwrite("field 1-2 edited.jpg", roi2)
#cv2.imwrite("field 1-2 binary.jpg", roi)

59 ms ± 572 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
